In [1]:
import requests
from kafka import KafkaProducer
import json
import logging

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Kafka configuration (update the broker if needed)
kafka_config = {
    'bootstrap_servers': 'kafka1:9092'
}

# Initialize a single Kafka Producer to be reused later
producer = KafkaProducer(
    bootstrap_servers=kafka_config['bootstrap_servers'],
    value_serializer=lambda v: json.dumps(v).encode('utf-8')
)


INFO:kafka.conn:<BrokerConnection node_id=bootstrap-0 host=kafka1:9092 <connecting> [IPv4 ('172.25.0.8', 9092)]>: connecting to kafka1:9092 [('172.25.0.8', 9092) IPv4]
INFO:kafka.conn:Probing node bootstrap-0 broker version
INFO:kafka.conn:<BrokerConnection node_id=bootstrap-0 host=kafka1:9092 <connecting> [IPv4 ('172.25.0.8', 9092)]>: Connection complete.
INFO:kafka.conn:Broker version identified as 2.6.0
INFO:kafka.conn:Set configuration api_version=(2, 6, 0) to skip auto check_version requests on startup


In [ ]:
def get_arrets():
    """
    Fetches all stops from the preprod arrets endpoint.
    Returns a list of stops.
    """
    arrets_api_url = "https://openv2-preprod.tan.fr/ewp/arrets.json"
    try:
        response = requests.get(arrets_api_url)
        response.raise_for_status()
    except requests.RequestException as e:
        logger.error(f"Failed to fetch stops: {e}")
        return []
    
    try:
        data = response.json()
    except json.JSONDecodeError:
        logger.error("Stops response is not valid JSON")
        return []
    
    # The response can be a list or a dict with a 'feeds' key.
    if isinstance(data, list):
        stops = data
    elif isinstance(data, dict):
        stops = data.get("feeds", [data])
    else:
        logger.error("Unexpected JSON structure for stops")
        stops = []
    
    return stops

def get_stop_by_name(stop_name, stops):
    """
    Searches for a stop with a given name (libellé) in the list of stops.
    The comparison is case-insensitive.
    Returns the first matching stop or None if not found.
    """
    stop_name_lower = stop_name.lower()
    for stop in stops:
        libelle = stop.get("libelle", "").lower()
        if stop_name_lower in libelle:
            return stop
    return None

def get_waiting_times_for_all_stops():
    """
    Dynamically retrieves waiting time records for all stops.
    For each stop from arrets.json, it iterates over its bus lines
    and for each bus line, iterates over both sens values ("1" and "2").
    It calls the waiting times endpoint for that stop and filters the records
    matching the current bus line and sens.
    Returns a list of all matching waiting time records.
    """
    stops = get_arrets()
    if not stops:
        logger.error("Unable to fetch stops from arrets.json.")
        return []
    
    all_records = []
    for stop in stops:
        codeLieu = stop.get("codeLieu")
        if not codeLieu:
            logger.warning(f"Stop missing codeLieu: {stop}")
            continue
        
        # Retrieve waiting times for the stop using its codeLieu
        waiting_time_api_url = f"http://open.tan.fr/ewp/tempsattente.json/{codeLieu}"
        try:
            response = requests.get(waiting_time_api_url)
            response.raise_for_status()
        except Exception as e:
            logger.error(f"Error calling {waiting_time_api_url} for stop {codeLieu}: {e}")
            continue
        
        try:
            waiting_times = response.json()
        except Exception as e:
            logger.error(f"Invalid JSON from waiting times endpoint for {codeLieu}: {e}")
            continue
        
        if not isinstance(waiting_times, list):
            waiting_times = [waiting_times]
        
        # Get the list of bus lines for the stop
        lignes = stop.get("ligne", [])
        if not lignes:
            logger.warning(f"No bus lines found for stop {stop.get('libelle')}")
            continue
        
        # For each bus line and for each sens value, filter the waiting times.
        for line in lignes:
            target_line = line.get("numLigne")
            if not target_line:
                continue
            for target_sens in ["1", "2"]:
                for wt in waiting_times:
                    try:
                        wt_sens = int(wt.get("sens", 0))
                    except:
                        wt_sens = 0
                    if wt_sens != int(target_sens):
                        continue
                    
                    # Check if the waiting time record matches the current bus line.
                    ligne_info = wt.get("ligne", {})
                    numLigne = ligne_info.get("numLigne", "").upper()
                    if numLigne == target_line.upper():
                        # Attach additional context info.
                        wt["stopInfo"] = stop
                        wt["targetLine"] = target_line
                        wt["targetSens"] = target_sens
                        all_records.append(wt)
    
    return all_records


In [ ]:
# Dynamically retrieve waiting time records for all stops.
matching_records = get_waiting_times_for_all_stops()

if matching_records:
    logger.info("Matching waiting time records found:")
    for rec in matching_records:
        # Optionally, log the record details (formatted)
        logger.info(json.dumps(rec, indent=2, ensure_ascii=False))
        # Publish the record to the Kafka topic "waiting_time_topic"
        producer.send("waiting_time_topic", value=rec)
    
    # Flush the Kafka producer to ensure all messages are sent.
    producer.flush()
    logger.info("Finished sending waiting time records to Kafka.")
else:
    logger.info("No waiting time records found for any stop.")


In [ ]:
def get_parkings():
    """
    Fetches parking availability records from the Nantes Métropole dataset endpoint.
    Returns a list of parking records.
    """
    parkings_api_url = "https://data.nantesmetropole.fr/api/explore/v2.1/catalog/datasets/244400404_parkings-publics-nantes-disponibilites/records"
    try:
        response = requests.get(parkings_api_url)
        response.raise_for_status()
    except requests.RequestException as e:
        logger.error(f"Failed to fetch parking data: {e}")
        return []
    
    try:
        data = response.json()
    except json.JSONDecodeError:
        logger.error("Parking response is not valid JSON")
        return []
    
    # The JSON response is expected to contain "total_count" and "results"
    records = data.get("results", [])
    logger.info(f"Fetched {len(records)} parking records (Total count: {data.get('total_count')}).")
    return records


In [ ]:
# Retrieve parking availability records
parking_records = get_parkings()

if parking_records:
    for record in parking_records:
        # Optionally, you can perform additional processing or transformation here.
        producer.send("parkings_topic", value=record)
        logger.info(f"Sent parking record: {json.dumps(record, indent=2, ensure_ascii=False)}")
    
    # Ensure all messages are sent before closing the producer
    producer.flush()
    logger.info("Finished sending parking records to Kafka.")
else:
    logger.info("No parking records found.")
